In [ ]:
import harmonica as hm
import numpy as np
import verde as vd

from magali._inversion import MagneticMomentBz, MagneticDipoleBz
from magali._synthetic import dipole_bz

from magali._units import (
    meter_to_micrometer,
    coordinates_micrometer_to_meter,
    nanotesla_to_tesla,
    tesla_to_nanotesla,
)

import matplotlib.pyplot as plt

In [2]:
dipole_coordinates = (500, 500, -15)
true_inclination = 30
true_declination = 40
true_intensity = 5e-11
true_moment = hm.magnetic_angles_to_vec(
    inclination=true_inclination,
    declination=true_declination,
    intensity=true_intensity,
)
coordinates = vd.grid_coordinates(
    region=[0, 100, 0, 100],
    spacing=1,
    extra_coords=5,
)

data = dipole_bz(coordinates, dipole_coordinates, true_moment)
model = MagneticDipoleBz((300, 130, -1) + (1.5e-11, 3e-11, -2e-11))

print(dipole_coordinates)
print(true_moment)
print(model.initial)

(500, 500, -15)
(np.float64(2.783351996132097e-11), np.float64(3.3170697408446925e-11), np.float64(-2.4999999999999998e-11))
[ 3.0e+02  1.3e+02 -1.0e+00  1.5e-11  3.0e-11 -2.0e-11]


In [3]:
xc, yc, zc = coordinates_micrometer_to_meter(dipole_coordinates)
m = nanotesla_to_tesla(1.5)
m 

1.5e-09

In [4]:
model.fit(coordinates, data)

In [5]:
model.location_

array([ 1263010.30660763,  2445671.76123956, -4937206.35297873])

In [6]:
model.dipole_moment_

array([-1859.41609074,  -389.21534501,   146.88000343])

In [ ]:
plt.imshow(model.jacobian)

NameError: name 'imshow' is not defined